In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
import json
from bayesian_predic__ import BayPredict

In [3]:
#Foi necessário pra eu replicar a base de dados categorizada pra utilizar na extimção das CPDS
with open('ind_004.json', 'r') as f:
    params = json.load(f)

N_YEARS = params['n_years']
NORM_TYPE = params['norm_type']
N_CAT = params['n_cat']
TARGET_VRIABLE = params['target_variable']
TOPOLOGY = params['topologia']
DATA_SOURCE = params['data_source']
STARTING_YEAR = params['starting_year']

pred = BayPredict(N_YEARS, NORM_TYPE, N_CAT, TARGET_VRIABLE, TOPOLOGY, DATA_SOURCE, STARTING_YEAR)

df = pred.DESEASONED_DF.pipe(pred.normalize_and_categorize_data)

df = df.iloc[:, :len(df.columns)//2]

df.columns = [column.split(':')[0] for column in df.columns]

In [4]:
## Caso a topologia esteja com as variáveis com os nomes de 0 a n como mostrado no cromossomo
from AG_funcs import get_random_topology_edges, VARIABLE_MAP

top = get_random_topology_edges(4,4, VARIABLE_MAP) #Só para gerat um exemplo

topology = []
for edge in top:
    topology.append([VARIABLE_MAP[edge[0]], VARIABLE_MAP[edge[1]]])

In [5]:
node_set = set({})
for edge in topology:
    for node in edge:
        node_set = node_set | {node}

In [6]:
bayesian_network = BayesianNetwork(topology)

In [7]:
#estimando as cpds baseando-se nos dados
list_cpds = [MaximumLikelihoodEstimator(bayesian_network, df).estimate_cpd(node) for node in node_set]

bayesian_network.add_cpds(*list_cpds)

In [9]:
topology

[['SIN-DM', 'TER'], ['SIN-DM', 'NUC'], ['Preco_L', 'TER'], ['TER', 'NUC']]

In [8]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(bayesian_network)

#variables recebe a lista de variáveis que se deseja observar
#evidence são as inferências, nesse caso, que a energia armazenada será estado 4
#tomar cuidado para as variáveis informadas estarem na rede

print(infer.query(variables=['TER'], evidence={'SIN-EA': 4}))

ValueError: Node SIN-EA not in not in graph